# Axial Annular Flow - Power Law

![caption](frederickson_bird_ax_ann_flow_fig.png?raw=1)

## Calculation of $\beta$ (or $\sigma$ in hanks and Larsen)

Solve for $\beta$ in

$\int_{\kappa}^{\beta}\left(\frac{beta^2}{\xi}-\xi\right)^{\frac{1}{n}}d\xi$ = $\int_{\beta}^{1}\left(\xi-\frac{beta^2}{\xi}\right)^{\frac{1}{n}}d\xi$

$\beta=\lambda$ from Hanks paper

$\sigma=\kappa=\frac{R_{Inner}}{R_{Outer}}$

n = power-law index

![caption](hanks_larsen_ax_ann_flow.png?raw=1)

In [1]:
from numpy import *
import numpy as np
from scipy.integrate import quad
from scipy.optimize import fsolve
import pandas as pd

In [2]:
# Axial annular flow for power-law fluid 2006
class beta():
    def betaeqn(self,beta):
        return self.intlhs(beta) - self.intrhs(beta)
    def lhs(self, beta, xi):
        return ( beta**2/xi - xi)**(1.0/self.n)
    def rhs(self,beta,xi):
        return (xi-beta**2/xi)**(1.0/self.n)
    def intlhs(self, beta):
        return quad(lambda xi: self.lhs(beta,xi),self.kappa, beta)[0]
    def intrhs(self, beta):
        return quad(lambda xi: self.rhs(beta,xi),beta, 1.0)[0]
    def qeqn(self, dplguess):
        self.dpl = dplguess
        beta = self.betacomp()
        return self.qdef - 3.14159265359*self.rad**3/(1.0/self.n+3.0)* \
                    (self.rad/(2.0*self.k)*dplguess)**(1.0/self.n)*((1.0-beta**2)** \
                    (1.0+1.0/self.n)-self.kappa**(1.0-1.0/self.n)*(beta**2-self.kappa**2) \
                    **(1.0+1.0/self.n))
    
class anncomp(beta):
    def betacomp(self):
        self.betaguess = self.kappa + (1.0-self.kappa)/2.0
        self.beta = fsolve( self.betaeqn, self.betaguess)
        return self.beta

In [3]:
# Generate list of kappa and n according to Hanks and Larsen
hanks_kappa = [0.05,0.08,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
hanks_n = np.linspace(0.1,1.0,19)

In [6]:
b = anncomp()
beta_dict = {}
for kappa_val in hanks_kappa:
    tmp_list=[]
    b.kappa = kappa_val
    for n_val in hanks_n:
        b.n=n_val
        tmp_list.append(b.betacomp()[0])
    beta_dict[kappa_val]=tmp_list
hanks_df = pd.DataFrame(beta_dict,index=hanks_n)

In [7]:
hanks_df

,0.05,0.08,0.10,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90
0.10,0.253339,0.311824,0.344175,0.468652,0.563164,0.643077,0.714020,0.778754,0.838864,0.895353,0.948903
0.15,0.267526,0.325066,0.356674,0.477631,0.569413,0.647290,0.716727,0.780368,0.839715,0.895709,0.948987
0.20,0.281081,0.337397,0.368177,0.485620,0.574886,0.650946,0.719064,0.781758,0.840447,0.896015,0.949059
0.25,0.293918,0.348815,0.378720,0.492744,0.579707,0.654145,0.721100,0.782966,0.841081,0.896281,0.949122
0.30,0.305990,0.359345,0.388360,0.499115,0.583977,0.656964,0.722890,0.784025,0.841637,0.896513,0.949177
0.35,0.317281,0.369033,0.397169,0.504833,0.587781,0.659465,0.724473,0.784961,0.842128,0.896718,0.949225
0.40,0.327799,0.377936,0.405217,0.509984,0.591186,0.661698,0.725884,0.785794,0.842565,0.896900,0.949268
0.45,0.337571,0.386115,0.412576,0.514640,0.594251,0.663702,0.727149,0.786541,0.842956,0.897063,0.949307
0.50,0.346634,0.393631,0.419314,0.518865,0.597022,0.665510,0.728289,0.787213,0.843308,0.897210,0.949341
0.55,0.355031,0.400545,0.425494,0.522712,0.599538,0.667150,0.729322,0.787821,0.843627,0.897343,0.949373


In [8]:
b.kappa=.08
b.n=.15
b.betacomp()[0]

0.3250655139018374